In [2]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.options.display.max_rows = 999
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier


In [3]:
FeatureTrain = pd.read_csv('./data/dengue_features_train.csv',parse_dates = ['week_start_date'])
TargetTrain = pd.read_csv('./data/dengue_labels_train.csv' )
FeatureTest = pd.read_csv('./data/dengue_features_test.csv',parse_dates = ['week_start_date'])
Answersheet = pd.read_csv('./data/submission_format.csv' )

### look at target first

In [4]:
"""
This is the difference between two measure

https://journals.ametsoc.org/doi/pdf/10.1175/BAMS-D-14-00226.1


"""

'\nThis is the difference between two measure\n\nhttps://journals.ametsoc.org/doi/pdf/10.1175/BAMS-D-14-00226.1\n\n\n'

In [5]:
panel = TargetTrain.merge(FeatureTrain,on = ['city','year','weekofyear'] ,how = 'outer') 
 
kelvin = ['reanalysis_air_temp_k', 'reanalysis_avg_temp_k',   'reanalysis_max_air_temp_k','reanalysis_min_air_temp_k','reanalysis_dew_point_temp_k']
panel.loc[:,kelvin] = panel.loc[:,kelvin]-273.15# kelvin to C
FeatureTest.loc[:,kelvin] =FeatureTest.loc[:,kelvin]-273.15# kelvin to C
panel.columns

timeid = ['year', 'weekofyear']
green = ['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw']
precipitation  =['precipitation_amt_mm','reanalysis_sat_precip_amt_mm','station_precip_mm','reanalysis_precip_amt_kg_per_m2',]
 
avg_temp = ['reanalysis_air_temp_k', 'reanalysis_avg_temp_k',  'station_avg_temp_c' ]
   
min_temp = ['station_min_temp_c','reanalysis_min_air_temp_k']
 
max_temp = ['station_max_temp_c','reanalysis_max_air_temp_k']
 
dtr =  ['reanalysis_tdtr_k', 'station_diur_temp_rng_c']
 

humid = ['reanalysis_dew_point_temp_k','reanalysis_specific_humidity_g_per_kg', 'reanalysis_relative_humidity_percent']
 
selected = humid + dtr + max_temp +min_temp +avg_temp +precipitation +timeid+green + ['city','total_cases','week_start_date']
features_selected= humid + dtr + max_temp +min_temp +avg_temp +precipitation +timeid+green  
# precipitation_amt_mm  & reanalysis_sat_precip_amt_mm are the same
 
panel = panel.loc[:,selected]
 

In [6]:
city_name = 'sj'
def get_data_panel(panel,city_name):
    panel_train =  panel.loc[panel.city == city_name].interpolate()  .set_index('week_start_date').copy().drop(['city'],axis = 1).copy()
    
    panel_test = FeatureTest.loc[FeatureTest.city == city_name].interpolate().set_index('week_start_date').copy().drop('city',axis = 1)
    panel_test['total_cases'] = np.nan 
    train_len = len(panel_test['total_cases'])
    panel = pd.concat([panel_train,panel_test],sort = False)
    
    
    panel['green_s']  = panel.loc[:,green[:2]].mean(1)
    panel['green_n']  = panel.loc[:,green[2:]].mean(1) 
    panel['green']  = panel.loc[:,green ].mean(1) 

    panel  = panel 
    return panel,train_len

panel_sj,len_sj = get_data_panel(panel,'sj')
panel_iq,len_iq = get_data_panel(panel,'iq')




In [7]:
from numpy import polyfit
def get_season(panel):
    degree = 3
 
    Y = panel .clip(0, panel .quantile(0.8))
    X = ( Y.index.weekofyear%52)

    bases  = []
    for i in reversed(range(degree+1)):
        bases.append(X**i)
    bases = pd.DataFrame(bases ).T
    params = polyfit( Y.dropna().index.weekofyear%52,Y.dropna(),deg = degree)


    return (bases .dot(params) *20) 
    
  


In [27]:

panel_sj_selected  = panel_sj.loc[:,['total_cases']]
panel_iq_selected  = panel_iq.loc[:,['total_cases']] 

panel_sj_selected['season'] = get_season(panel_sj.loc[:,'total_cases'] ) .ravel()
panel_sj_selected['season'] = panel_sj_selected['season'] .clip(0,panel_sj_selected['season'] .quantile(0.9 ))
panel_iq_selected['season'] = get_season(panel_iq.loc[:,'total_cases'] ) .ravel()
panel_iq_selected['season'] = panel_iq_selected['season'] .clip(0,panel_iq_selected['season'] .quantile(0.9 ))
 
def append_shift(left,right,selected,shift,smooth =52):
    new_names = []
    right = right.copy().apply(lambda x : x.clip(0,x.quantile(0.9)),axis = 0)
    for name in selected:
        new_name = name + '_shift_' +str(shift)+'_smooth_' + str(smooth)
        new_names.append(new_name)
 
        left[new_name] = right.rolling(smooth    ).mean().shift(shift).loc[:,name ]
    return left


def append_product(left,right, selected,shift,smooth = 52):
    new_name = selected[0] + "_X_" + selected[1]
    left[new_name]       = right.loc[:,selected[0] ].multiply (right.loc[:,selected[1] ],axis = 0).rolling(smooth ).mean()
    return left
 
def data_process(panel,panel_candidate):
    
    """
    Make sure it is at least forward filled
    """
    
    panel = append_shift(panel,panel_candidate,\
          ['reanalysis_specific_humidity_g_per_kg',
            'reanalysis_sat_precip_amt_mm',\
           'station_min_temp_c'  ],0,52)
    
    panel = append_shift(panel,panel_candidate,\
          ['reanalysis_specific_humidity_g_per_kg',
            'reanalysis_sat_precip_amt_mm',\
           'station_min_temp_c','green_n' ],0,20)
 
    
    return panel 


panel_sj_selected =  data_process(panel_sj_selected,panel_sj)
panel_iq_selected =  data_process(panel_iq_selected,panel_iq) 

## Cross validation

In [28]:
 

train_y_sj, train_X_sj =   panel_sj_selected .dropna().loc[:,'total_cases'] .iloc[: -len_sj],\
                            panel_sj_selected.dropna() .drop('total_cases',axis = 1) .iloc[: -len_sj]  
 
train_y_iq, train_X_iq =   panel_iq_selected.dropna() .loc[:,'total_cases'] .iloc[60: -len_iq],\
                            panel_iq_selected.dropna() .drop('total_cases',axis = 1) .iloc[60: -len_iq]  
                         

test_X_sj = panel_sj_selected.drop('total_cases',axis = 1)  .iloc[-len_sj: ]
test_X_iq = panel_iq_selected.drop('total_cases',axis = 1)  .iloc[-len_iq: ]

In [31]:
mdl_sj = RandomForestRegressor(n_estimators=400,  
                                         max_depth=6, criterion='mae',min_samples_split = 8,  warm_start=True,max_samples = 0.5,\
                               min_samples_leaf = 4)
mdl_sj.fit( train_X_sj ,train_y_sj.clip(0,200) )
mdl_iq = RandomForestRegressor(n_estimators=100, max_features='auto',
                                         max_depth=6, criterion='mae', min_weight_fraction_leaf=0.1, warm_start=True)

mdl_iq.fit(train_X_iq  ,train_y_iq.clip(0,40))


sj_predictions =  pd.Series(np.round(mdl_sj . predict( test_X_sj)).astype(int) ) 
iq_predictions =  pd.Series(np.round(mdl_iq .predict(test_X_iq )).astype(int) ) 
    
 
    
submission = pd.read_csv("./data/submission_format.csv", index_col=[0, 1, 2])
submission.total_cases =np.concatenate([sj_predictions, iq_predictions])
 
submission.to_csv("./data/submission_012800.csv")

In [45]:
test_X_sj.iloc[:,3].name

'reanalysis_sat_precip_amt_mm_shift_0_smooth_52'